In [1]:
import pymongo
from pymongo import MongoClient, GEOSPHERE
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv
import os
import requests
import json
from functools import reduce
import operator
import geopandas as gpd
import cartoframes
from cartoframes.viz import Map, Layer, popup_element
import shapely.geometry

# Office location analysis

#### We are going to select a possible location for the office based on the available information by using Mongo db.

The applied filter assums a **number of emplooyees of 90**. 

The reason? Currently, a huge number of companies have implanted a ***work from home policy*** but, additionally the company **expect to increase their employees number in the future**, so being continously changing the location would be counterproductive.

In [2]:
client = MongoClient()
db = client.companies

NYC is the second city worldwide and the first city in the USA in ***number of Starbucks***. For this reason, we have selected ***New York City*** as the city to establish our office.

In [3]:
filt1 = {"number_of_employees": {"$eq":90},"offices.city": {'$eq': "New York"}}
project1 = {"_id":0,"name":1,"offices.city": 1,"number_of_employees": 1,"offices.latitude":1,"offices.longitude":1}
results1 = db.offices.find(filt1,project1).sort("number_of_employees",1)
list(results1)

[{'name': 'Aleri',
  'number_of_employees': 90,
  'offices': {'city': 'New York',
   'latitude': 40.7592189,
   'longitude': -73.9783534}}]

### Office location

Our office is finally located where Aleri company were based on NYC. The coordenates are:

In [4]:
office_lat = 40.7592189
office_long = -73.9783534
print (office_lat,office_long)

40.7592189 -73.9783534


- **Latitud** = 40.7592189

- **Longitud** = -73.9783534

In [5]:
# List that contains the information about office location.
office_data = [("office","40.7592189","-73.9783534","POINT (-73.9783534 40.7592189)","office location")]
df_office = pd.DataFrame(office_data, columns = ["name","latitud","longitud","geometry","criteria"]) 
df_office

,name,latitud,longitud,geometry,criteria
0,office,40.7592189,-73.9783534,POINT (-73.9783534 40.7592189),office location


### Visualization of the location on a map.

In [6]:
map_1 = folium.Map(location = [office_lat, office_long], zoom_start = 15)
office_loc = folium.Marker(location = [office_lat, office_long], tooltip = "Office location proposal")
icono = Icon(color = "blue",
             prefix = "fa",
             icon = "building-o",
             icon_color = "black",
             tooltip = "Office location proposal")

In [7]:
# We apply the previously defined variables in the marker function.
office_location = [office_lat, office_long]
marker_office = Marker(location = office_location, icon = icono)
marker_office.add_to(map_1)
map_1

In [8]:
map_1.save("map_1.html")

## Once the location has been selected, we are looking for the criterias.

In [9]:
# Checking that Foursquare's tokens are available and defining them.
load_dotenv()
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [10]:
# Definition of a variable called office_location with the coordinates of the office location, as we want to search around it.
office_location = {'type': 'Point', 'coordinates': [40.7592189, -73.9783534]}
print (office_location)

{'type': 'Point', 'coordinates': [40.7592189, -73.9783534]}


### 1. Starbucks

A 11.5% of the employees satisfies with it.

In [11]:
# By taking information from Foursquare's API, we are looking for Starbucks around our office in a radius of 500m.

url_query = 'https://api.foursquare.com/v2/venues/explore'
starbucks = "556f676fbd6a75a99038d8ec"

parameters = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{office_location.get('coordinates')[0]},{office_location.get('coordinates')[1]}",
              "query":f"Starbucks",
              "radius":500}

In [12]:
resp = requests.get(url= url_query, params = parameters)
data = json.loads(resp.text)
list(data)

['meta', 'response']

In [13]:
decoding_data = data.get("response")
decoded = decoding_data.get("groups")[0]
starbucks = decoded.get("items")

In [14]:
map_starbucks = ["venue","name"]
m_latitude = ["venue","location","lat"]
m_longitude = ["venue","location","lng"]

In [15]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

In [16]:
print(getFromDict(starbucks[0],map_starbucks))

Starbucks


In [17]:
starbucks_nyc = []
for dic in starbucks:
    list1 = {}
    list1["name"] = getFromDict(dic,map_starbucks)
    list1["latitud"] = getFromDict(dic,m_latitude)
    list1["longitud"] = getFromDict(dic,m_longitude)
    starbucks_nyc.append(list1)

In [18]:
# Conversion on dataframe that are used on maps construction.
df_starbucks = pd.DataFrame(starbucks_nyc)
df_starbucks.head()

,name,latitud,longitud
0,Starbucks,40.758149,-73.979420
1,Starbucks,40.759535,-73.980981
2,Starbucks,40.760602,-73.976467
3,Starbucks,40.761845,-73.978700
4,Starbucks,40.756665,-73.974397


#### A sum of 20 Starbucks are located in a radius of 500m from the office.

In [19]:
#### Finally, we convert our data frame into a geodataframe.

gdf_starbucks = gpd.GeoDataFrame(df_starbucks, geometry = gpd.points_from_xy(df_starbucks.longitud,df_starbucks.latitud))

In [20]:
### Creating the column criteria
gdf_starbucks = gdf_starbucks.assign(name = [ "Starbucks"]*20,
               criteria = ["Starbucks"]*20)
gdf_starbucks.head()

,name,latitud,longitud,geometry,criteria
0,Starbucks,40.758149,-73.979420,POINT (-73.97942 40.75815),Starbucks
1,Starbucks,40.759535,-73.980981,POINT (-73.98098 40.75953),Starbucks
2,Starbucks,40.760602,-73.976467,POINT (-73.97647 40.76060),Starbucks
3,Starbucks,40.761845,-73.978700,POINT (-73.97870 40.76185),Starbucks
4,Starbucks,40.756665,-73.974397,POINT (-73.97440 40.75666),Starbucks


In [21]:
# Convert our dataframe on a dict to be able to use it in MongoDB.
gdf_starbucks["geometry"] = gdf_starbucks["geometry"].apply(lambda x: shapely.geometry.mapping(x))
gdf_starbucks[:5]

/usr/local/lib/python3.9/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,name,latitud,longitud,geometry,criteria
0,Starbucks,40.758149,-73.979420,"{'type': 'Point', 'coordinates': (-73.97941981...",Starbucks
1,Starbucks,40.759535,-73.980981,"{'type': 'Point', 'coordinates': (-73.98098084...",Starbucks
2,Starbucks,40.760602,-73.976467,"{'type': 'Point', 'coordinates': (-73.97646651...",Starbucks
3,Starbucks,40.761845,-73.978700,"{'type': 'Point', 'coordinates': (-73.9787, 40...",Starbucks
4,Starbucks,40.756665,-73.974397,"{'type': 'Point', 'coordinates': (-73.97439689...",Starbucks


In [22]:
type(gdf_starbucks.loc[0,"geometry"])

dict

In [23]:
client = MongoClient() 
db = client.companies
collection = db.starbucks 
collection.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [24]:
data = gdf_starbucks.to_dict(orient='records')
data[0]

{'name': 'Starbucks',
 'latitud': 40.75814852475969,
 'longitud': -73.97941981951507,
 'geometry': {'type': 'Point',
  'coordinates': (-73.97941981951507, 40.75814852475969)},
 'criteria': 'Starbucks'}

#### Finding the nearest Starbucks

In [25]:
collection.insert_many(data)

In [26]:
filt = {"geometry":{"$near":{"type":"Point","coordinates":[40.7592189, -73.9783534]}}}
proyect = {"_id":0,"name":1,"latitud":1,"longitud":1}

result = db.starbucks.find(filt,proyect).limit(1)
starbucks = list(result)

In [27]:
starbucks

[{'name': 'Starbucks',
  'latitud': 40.756086423412924,
  'longitud': -73.9749708319223}]

### 2. Vegan restaurants
The CEO of the company is satisfied with this election.

In [28]:
url2 = 'https://api.foursquare.com/v2/venues/explore'
vegan = "4bf58dd8d48988d1d3941735"

In [29]:
# By taking information from Foursquare's API, we are looking for vegan restaurants around our office in a radius of 500m.
parameters_vegan = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{office_location.get('coordinates')[0]},{office_location.get('coordinates')[1]}",
              "query":f"vegans",
                "radius":500
}

In [30]:
resp2 = requests.get(url = url_query, params = parameters_vegan)
data2 = json.loads(resp2.text)

In [31]:
decoding_data2 = data2.get("response")
decoded2 = decoding_data2.get("groups")[0]
vegan = decoded2.get("items")

In [32]:
map_vegan = ["venue","name"]
m_latitudvegan = ["venue","location","lat"]
m_longitudvegan = ["venue","location","lng"]

In [33]:
def getFromDict2(diccionario2,mapa2):
    return reduce (operator.getitem,mapa2,diccionario2)

In [34]:
vegan_nyc = []
for dic in vegan:
    paralista2 = {}
    paralista2["name"] = getFromDict(dic,map_vegan)
    paralista2["latitud"] = getFromDict(dic,m_latitudvegan)
    paralista2["longitud"] = getFromDict(dic,m_longitudvegan)
    vegan_nyc.append(paralista2)

In [35]:
df_vegans = pd.DataFrame(vegan_nyc)
df_vegans.head()

,name,latitud,longitud
0,Beyond Sushi,40.763454,-73.977163
1,by CHLOE.,40.758063,-73.978854
2,Van Leeuwen Ice Cream,40.759578,-73.980338
3,Taco Dumbo,40.760295,-73.977430
4,L’Avenue,40.758756,-73.977673


**30 Vegan restaurants** are located in a radius of 500m from the office.

In [36]:
#### Finally, we convert our data frame into a geodataframe.

gdf_vegan = gpd.GeoDataFrame(df_vegans, geometry = gpd.points_from_xy(df_vegans.longitud,df_vegans.latitud))
gdf_vegan.head()

,name,latitud,longitud,geometry
0,Beyond Sushi,40.763454,-73.977163,POINT (-73.97716 40.76345)
1,by CHLOE.,40.758063,-73.978854,POINT (-73.97885 40.75806)
2,Van Leeuwen Ice Cream,40.759578,-73.980338,POINT (-73.98034 40.75958)
3,Taco Dumbo,40.760295,-73.977430,POINT (-73.97743 40.76029)
4,L’Avenue,40.758756,-73.977673,POINT (-73.97767 40.75876)


In [37]:
# Adding a criteria column
gdf_vegan = gdf_vegan.assign(name = ["Beyond Sushi",
                                        "by CHLOE.",
                                        "Van Leeuwen Ice Cream",
                                        "Taco Dumbo",
                                        "L’Avenue",
                                        "Taco Dumbo",
                                        "Urbanspace W52",
                                        "The Halal Guys",
                                        "The Little Beet",
                                        "Lenwich by Lenny's",
                                        "Le Pain Quotidien",
                                        "Mysttik Masala",
                                        "Museum of Modern Art (MoMA)",
                                        "Fogo de Chão",
                                        "Aldo Sohm Wine Bar",
                                        "Gregorys Coffee",
                                        "Pret A Manger",
                                        "Devon & Blakely",
                                        "Fig & Olive",
                                        "The Modern",
                                        "Black Tap",
                                        "Simon Sips",
                                        "Barilla Restaurants",
                                        "Estiatorio Milos",
                                        "Le Pain Quotidien",
                                        "Cock & Bull British Pub and Eatery",
                                        "Europa Cafe",
                                        "Forty2West",
                                        "Butter Midtown",
                                        "Natureworks"],
                                criteria = ["vegan"]*30)
gdf_vegan.head()

,name,latitud,longitud,geometry,criteria
0,Beyond Sushi,40.763454,-73.977163,POINT (-73.97716 40.76345),vegan
1,by CHLOE.,40.758063,-73.978854,POINT (-73.97885 40.75806),vegan
2,Van Leeuwen Ice Cream,40.759578,-73.980338,POINT (-73.98034 40.75958),vegan
3,Taco Dumbo,40.760295,-73.977430,POINT (-73.97743 40.76029),vegan
4,L’Avenue,40.758756,-73.977673,POINT (-73.97767 40.75876),vegan


In [38]:
gdf_vegan["geometry"] = gdf_vegan["geometry"].apply(lambda x: shapely.geometry.mapping(x))
gdf_vegan[:5]

/usr/local/lib/python3.9/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,name,latitud,longitud,geometry,criteria
0,Beyond Sushi,40.763454,-73.977163,"{'type': 'Point', 'coordinates': (-73.97716274...",vegan
1,by CHLOE.,40.758063,-73.978854,"{'type': 'Point', 'coordinates': (-73.9788542,...",vegan
2,Van Leeuwen Ice Cream,40.759578,-73.980338,"{'type': 'Point', 'coordinates': (-73.980338, ...",vegan
3,Taco Dumbo,40.760295,-73.977430,"{'type': 'Point', 'coordinates': (-73.97743043...",vegan
4,L’Avenue,40.758756,-73.977673,"{'type': 'Point', 'coordinates': (-73.97767334...",vegan


In [39]:
type(gdf_vegan.loc[0,"geometry"])

dict

In [40]:
# Export our query
client = MongoClient() 
db = client.companies
collection = db.vegan 
collection.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [41]:
data = gdf_vegan.to_dict(orient='records')
data[0]

{'name': 'Beyond Sushi',
 'latitud': 40.76345369164551,
 'longitud': -73.9771627466244,
 'geometry': {'type': 'Point',
  'coordinates': (-73.9771627466244, 40.76345369164551)},
 'criteria': 'vegan'}

In [42]:
collection.insert_many(data)

In [43]:
# Determine the nearest vegan restaurantt

In [44]:
filt = {"geometry":{"$near":{"type":"Point","coordinates":[40.7592189, -73.9783534]}}}
proyect = {"_id":0,"name":1,"latitud":1,"longitud":1}

result = db.vegan.find(filt,proyect).limit(1)
vegan = list(result)

In [45]:
vegan

[{'name': 'Devon & Blakely',
  'latitud': 40.75560132228682,
  'longitud': -73.97619031651554}]

### 3. Basketball stadiums.

The satisfaction of the maintenance guy would be so high if there is any basketball stadium located in a radius of 10km as maximum.

In [46]:
url3 = 'https://api.foursquare.com/v2/venues/explore'
basketball_stadium = "4bf58dd8d48988d18b941735"

In [47]:
# By taking information from Foursquare's API, we are looking for basketball stadiums around our office in a radius of 1000m.

parametros_basket = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{office_location.get('coordinates')[0]},{office_location.get('coordinates')[1]}",
              "query":f"basket_stadium",
                "radius":1000
}

In [48]:
resp3 = requests.get(url = url_query, params = parametros_basket)
data3 = json.loads(resp3.text)

In [49]:
decoding_data3 = data3.get("response")
decoded3 = decoding_data3.get("groups")[0]
basket_stadium = decoded3.get("items")

In [50]:
map_stadium = ["venue","name"]
m_latitud_stadium = ["venue","location","lat"]
m_longitud_stadium = ["venue","location","lng"]
def getFromDict(diccionario3,mapa3):
    return reduce (operator.getitem,mapa3,diccionario3)

In [51]:
basket_stadium_nyc = []
for dic in basket_stadium:
    paralista3 = {}
    paralista3["name"] = getFromDict(dic,map_stadium)
    paralista3["latitud"] = getFromDict(dic,m_latitud_stadium)
    paralista3["longitud"] = getFromDict(dic,m_longitud_stadium)
    basket_stadium_nyc.append(paralista3)

In [52]:
df_stadium = pd.DataFrame(basket_stadium_nyc)
df_stadium.head()

,name,latitud,longitud
0,Regal E-Walk 4DX & RPX,40.756960,-73.989286
1,Zaro's Bakery,40.752875,-73.976946
2,Boomer Esiason's Stadium Grill,40.757840,-73.987099
3,Stadium Grill At Bowlmor Lanes,40.757211,-73.987395


**4 basketball stadiums** are located in a radius of 1000m. **Up to 22 increasing** the radius to 10km.

In [53]:
# We convert our original dataframe into a geodataframe and we add a criteria column.
gdf_stadiums = gpd.GeoDataFrame(df_stadium, geometry = gpd.points_from_xy(df_stadium.longitud,df_stadium.latitud))
gdf_stadiums = gdf_stadiums.assign(name = [ "Regal E-Walk 4DX & RPX",
                           "Zaro's Bakery",
                           "Boomer Esiason's Stadium Grill",
                           "Stadium Grill At Bowlmor Lanes"],
               criteria = ["basketball stadium","basketball stadium","basketball stadium","basketball stadium"])
gdf_stadiums.head()

,name,latitud,longitud,geometry,criteria
0,Regal E-Walk 4DX & RPX,40.756960,-73.989286,POINT (-73.98929 40.75696),basketball stadium
1,Zaro's Bakery,40.752875,-73.976946,POINT (-73.97695 40.75288),basketball stadium
2,Boomer Esiason's Stadium Grill,40.757840,-73.987099,POINT (-73.98710 40.75784),basketball stadium
3,Stadium Grill At Bowlmor Lanes,40.757211,-73.987395,POINT (-73.98739 40.75721),basketball stadium


#### Export the query to Mongo

In [54]:
gdf_stadiums["geometry"] = gdf_stadiums["geometry"].apply(lambda x: shapely.geometry.mapping(x))
gdf_stadiums

/usr/local/lib/python3.9/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,name,latitud,longitud,geometry,criteria
0,Regal E-Walk 4DX & RPX,40.756960,-73.989286,"{'type': 'Point', 'coordinates': (-73.98928566...",basketball stadium
1,Zaro's Bakery,40.752875,-73.976946,"{'type': 'Point', 'coordinates': (-73.976946, ...",basketball stadium
2,Boomer Esiason's Stadium Grill,40.757840,-73.987099,"{'type': 'Point', 'coordinates': (-73.98709862...",basketball stadium
3,Stadium Grill At Bowlmor Lanes,40.757211,-73.987395,"{'type': 'Point', 'coordinates': (-73.98739450...",basketball stadium


In [55]:
type(gdf_stadiums.loc[0,"geometry"])

dict

In [56]:
client = MongoClient() 
db = client.companies
collection = db.basket_stadium 
collection.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [57]:
data = gdf_stadiums.to_dict(orient='records')
data[0]

{'name': 'Regal E-Walk 4DX & RPX',
 'latitud': 40.756960133814175,
 'longitud': -73.9892856652269,
 'geometry': {'type': 'Point',
  'coordinates': (-73.9892856652269, 40.756960133814175)},
 'criteria': 'basketball stadium'}

In [58]:
collection.insert_many(data)

#### Deterimining the nearest basket stadium

In [60]:
db.basket_stadium

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'basket_stadium')

In [61]:
filt = {"geometry":{"$near":{"type":"Point","coordinates":[40.7592189, -73.9783534]}}}
proyect = {"_id":0,"name":1,"latitud":1,"longitud":1}

result = db.basket_stadium.find(filt,proyect).limit(1)
basket_stadium = list(result)

In [62]:
basket_stadium

[{'name': "Zaro's Bakery", 'latitud': 40.752875, 'longitud': -73.976946}]

## 4. Night clubs

In [63]:
url4 = 'https://api.foursquare.com/v2/venues/explore'
night_clubs = "4bf58dd8d48988d11f941735"

In [64]:
# By taking information from Foursquare's API, we are looking for night clubs around our office in a radius of 1000m.
parametros_night_clubs = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{office_location.get('coordinates')[0]},{office_location.get('coordinates')[1]}",
              "query":f"night_clubs","radius":1000}

In [65]:
resp4 = requests.get(url = url_query, params = parametros_night_clubs)
data4 = json.loads(resp4.text)

In [66]:
decoding_data4 = data4.get("response")
decoded4 = decoding_data4.get("groups")[0]
night_clubs = decoded4.get("items")

In [67]:
map_night_clubs = ["venue","name"]
m_latitud_nightclubs = ["venue","location","lat"]
m_longitud_nightclubs = ["venue","location","lng"]

In [68]:
def getFromDic4t(diccionario4,mapa4):
    return reduce (operator.getitem,mapa4,diccionario4)

In [69]:
night_clubs_nyc = []
for dic in night_clubs:
    paralista4 = {}
    paralista4["name"] = getFromDict(dic,map_night_clubs)
    paralista4["latitud"] = getFromDict(dic,m_latitud_nightclubs)
    paralista4["longitud"] = getFromDict(dic,m_longitud_nightclubs)
    night_clubs_nyc.append(paralista4)

In [70]:
df_night_clubs = pd.DataFrame(night_clubs_nyc)
df_night_clubs.head()

,name,latitud,longitud
0,Lavo,40.762919,-73.971338
1,Studio 54,40.763944,-73.983980
2,JOE & THE JUICE,40.765877,-73.981975
3,Tao,40.762659,-73.971441
4,STK Steakhouse Midtown NYC,40.755262,-73.982888


A total amount of **11 night clubs** in a radius of 1000m from the office.

In [71]:
night_clubs_location = list(df_night_clubs)
gdf_night_clubs = gpd.GeoDataFrame(df_night_clubs, geometry = gpd.points_from_xy(df_night_clubs.longitud,df_night_clubs.latitud))
gdf_night_clubs.head()

,name,latitud,longitud,geometry
0,Lavo,40.762919,-73.971338,POINT (-73.97134 40.76292)
1,Studio 54,40.763944,-73.983980,POINT (-73.98398 40.76394)
2,JOE & THE JUICE,40.765877,-73.981975,POINT (-73.98197 40.76588)
3,Tao,40.762659,-73.971441,POINT (-73.97144 40.76266)
4,STK Steakhouse Midtown NYC,40.755262,-73.982888,POINT (-73.98289 40.75526)


In [72]:
# We convert our original dataframe into a geodataframe and we add a criteria column.

gdf_night_clubs = gpd.GeoDataFrame(df_night_clubs, geometry = gpd.points_from_xy(df_night_clubs.longitud,df_night_clubs.latitud))
gdf_night_clubs = gdf_night_clubs.assign(name = ["Lavo",
                                        "Studio 54",
                                        "JOE & THE JUICE",
                                        "Tao",
                                        "STK Steakhouse Midtown NYC",
                                        "The Smith",
                                        "The View Restaurant & Lounge",
                                        "Chivas 1801 Club- NYC",
                                        "Flowers by Richard",
                                        "The Mean Fiddler",
                                        "Public House"],
                    criteria = ["night_clubs"]*11)
gdf_night_clubs.head()

,name,latitud,longitud,geometry,criteria
0,Lavo,40.762919,-73.971338,POINT (-73.97134 40.76292),night_clubs
1,Studio 54,40.763944,-73.983980,POINT (-73.98398 40.76394),night_clubs
2,JOE & THE JUICE,40.765877,-73.981975,POINT (-73.98197 40.76588),night_clubs
3,Tao,40.762659,-73.971441,POINT (-73.97144 40.76266),night_clubs
4,STK Steakhouse Midtown NYC,40.755262,-73.982888,POINT (-73.98289 40.75526),night_clubs


#### Convert our dataframe on a dict to be able to use it in MongoDB.

In [73]:
gdf_night_clubs["geometry"] = gdf_night_clubs["geometry"].apply(lambda x: shapely.geometry.mapping(x))
gdf_night_clubs

/usr/local/lib/python3.9/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,name,latitud,longitud,geometry,criteria
0,Lavo,40.762919,-73.971338,"{'type': 'Point', 'coordinates': (-73.97133777...",night_clubs
1,Studio 54,40.763944,-73.983980,"{'type': 'Point', 'coordinates': (-73.98398019...",night_clubs
2,JOE & THE JUICE,40.765877,-73.981975,"{'type': 'Point', 'coordinates': (-73.98197482...",night_clubs
3,Tao,40.762659,-73.971441,"{'type': 'Point', 'coordinates': (-73.97144107...",night_clubs
4,STK Steakhouse Midtown NYC,40.755262,-73.982888,"{'type': 'Point', 'coordinates': (-73.98288809...",night_clubs
5,The Smith,40.755376,-73.968243,"{'type': 'Point', 'coordinates': (-73.96824326...",night_clubs
6,The View Restaurant & Lounge,40.758690,-73.986285,"{'type': 'Point', 'coordinates': (-73.986285, ...",night_clubs
7,Chivas 1801 Club- NYC,40.759405,-73.969916,"{'type': 'Point', 'coordinates': (-73.969916, ...",night_clubs
8,Flowers by Richard,40.764334,-73.985642,"{'type': 'Point', 'coordinates': (-73.985642, ...",night_clubs
9,The Mean Fiddler,40.760313,-73.987162,"{'type': 'Point', 'coordinates': (-73.98716191...",night_clubs


#### Export the query to Mongo

In [74]:
client = MongoClient() 
db = client.companies
collection = db.night_clubs 
collection.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [75]:
data = gdf_night_clubs.to_dict(orient='records')
data[0]

{'name': 'Lavo',
 'latitud': 40.762919168049976,
 'longitud': -73.97133777386868,
 'geometry': {'type': 'Point',
  'coordinates': (-73.97133777386868, 40.762919168049976)},
 'criteria': 'night_clubs'}

In [76]:
collection.insert_many(data)

#### Determine the nearest night club.

In [77]:
db.night_clubs 

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'night_clubs')

In [78]:
filt = {"geometry":{"$near":{"type":"Point","coordinates":[40.7592189, -73.9783534]}}}
proyect = {"_id":0,"name":1,"latitud":1,"longitud":1}

result = db.night_clubs .find(filt,proyect).limit(1)
night_clubs = list(result)

In [79]:
#Neares nightt club
night_clubs

[{'name': 'Public House',
  'latitud': 40.75055411817874,
  'longitud': -73.97603942215522}]

## 5. Transports availability.
The 23% of the employees are glad because of taking in consideration this criteria.

In [80]:
url5 = 'https://api.foursquare.com/v2/venues/explore'
travel_transport = "4d4b7105d754a06379d81259"

In [81]:
# By taking information from Foursquare's API, we are looking for transport availability.
parametros_travel = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{office_location.get('coordinates')[0]},{office_location.get('coordinates')[1]}",
              "query":f"travel_transport",
                ## "radius":1000
                    }

In [82]:
resp5 = requests.get(url = url_query, params = parametros_travel)
data5 = json.loads(resp5.text)
decoding_data5 = data5.get("response")
decoded5 = decoding_data5.get("groups")[0]
transport = decoded5.get("items")

In [83]:
map_transport = ["venue","name"]
m_latitud_transport = ["venue","location","lat"]
m_longitud_transport = ["venue","location","lng"]

In [84]:
def getFromDict(diccionario5,mapa5):
    return reduce (operator.getitem,mapa5,diccionario5)

In [85]:
transport_nyc = []
for dic in transport:
    paralista5 = {}
    paralista5["name"] = getFromDict(dic,map_stadium)
    paralista5["latitud"] = getFromDict(dic,m_latitud_stadium)
    paralista5["longitud"] = getFromDict(dic,m_longitud_stadium)
    transport_nyc.append(paralista5)

In [86]:
df_transports = pd.DataFrame(transport_nyc)
df_transports.head()

,name,latitud,longitud
0,Theater Row - The Acorn,40.758551,-73.993419
1,Solstice Travel Vacations,40.758305,-73.967880
2,The Travel Inn,40.760124,-73.996154
3,United Bus and Travel,40.713707,-73.991714
4,Double Happiness Travel,40.713707,-73.991714


**9 transports points** close to the office that facilitates travel for Account Managers.

In [87]:
# We convert our original dataframe into a geodataframe and we add a criteria column.

gdf_transports = gpd.GeoDataFrame(df_transports, geometry = gpd.points_from_xy(df_transports.longitud,df_transports.latitud))

gdf_transports = gdf_transports.assign(name = [ "Theater Row - The Acorn",
                           "Solstice Travel Vacations",
                           "The Travel Inn",
                           "United Bus and Travel",
                           "Double Happiness Travel",
                           "K International Transport Co., Inc",
                           "Delgado Travel",
                           "Active Transport ServicesInc.",
                           "Rivas Travel & Multi Services"],
               criteria = ["transports"]*9)
gdf_transports

,name,latitud,longitud,geometry,criteria
0,Theater Row - The Acorn,40.758551,-73.993419,POINT (-73.99342 40.75855),transports
1,Solstice Travel Vacations,40.758305,-73.967880,POINT (-73.96788 40.75830),transports
2,The Travel Inn,40.760124,-73.996154,POINT (-73.99615 40.76012),transports
3,United Bus and Travel,40.713707,-73.991714,POINT (-73.99171 40.71371),transports
4,Double Happiness Travel,40.713707,-73.991714,POINT (-73.99171 40.71371),transports
5,"K International Transport Co., Inc",40.708588,-74.011889,POINT (-74.01189 40.70859),transports
6,Delgado Travel,40.732078,-74.066660,POINT (-74.06666 40.73208),transports
7,Active Transport ServicesInc.,40.673784,-73.982472,POINT (-73.98247 40.67378),transports
8,Rivas Travel & Multi Services,40.852265,-73.934917,POINT (-73.93492 40.85227),transports


In [88]:
# Export to MongoDB

client = MongoClient() 
db = client.companies
collection = db.transports 
collection.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [89]:
data = gdf_transports.to_dict(orient='records')
data[0]

{'name': 'Theater Row - The Acorn',
 'latitud': 40.7585514688034,
 'longitud': -73.9934189068339,
 'geometry': <shapely.geometry.point.Point at 0x13b9e55b0>,
 'criteria': 'transports'}

#### Loking for the closet transport point.

In [90]:
db.transports 

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'transports')

In [91]:
filt = {"geometry":{"$near":{"type":"Point","coordinates":[40.7592189, -73.9783534]}}}
proyect = {"_id":0,"name":1,"latitud":1,"longitud":1}

result = db.transports .find(filt,proyect).limit(1)
transports = list(result)

In [92]:
# Closet point
transports

[{'name': 'Active Transport ServicesInc.',
  'latitud': 40.67378371119131,
  'longitud': -73.98247150733843}]

## Finally, we concatenate our criteria's dataframes.

In [93]:
df_criteria = pd.concat([df_office,gdf_starbucks, gdf_vegan,gdf_stadiums,gdf_night_clubs,gdf_transports])
df_criteria.head()

,name,latitud,longitud,geometry,criteria
0,office,40.7592189,-73.9783534,POINT (-73.9783534 40.7592189),office location
0,Starbucks,40.758149,-73.979420,"{'type': 'Point', 'coordinates': (-73.97941981...",Starbucks
1,Starbucks,40.759535,-73.980981,"{'type': 'Point', 'coordinates': (-73.98098084...",Starbucks
2,Starbucks,40.760602,-73.976467,"{'type': 'Point', 'coordinates': (-73.97646651...",Starbucks
3,Starbucks,40.761845,-73.978700,"{'type': 'Point', 'coordinates': (-73.9787, 40...",Starbucks


In [94]:
df_criteria.criteria.unique()

array(['office location', 'Starbucks', 'vegan', 'basketball stadium',
       'night_clubs', 'transports'], dtype=object)

## Maps

## Heat map

In [95]:
heat_map = folium.Map (location = [40.7592189,-73.9783534],zoom_start = 15)

In [96]:
criteria1 = df_criteria[df_criteria.criteria == "Starbucks"]
criteria1_group = folium.FeatureGroup(name = "Starbucks")
HeatMap(data = criteria1[["latitud","longitud"]],radius = 15).add_to(criteria1_group)
criteria1_group.add_to(heat_map)

criteria2 = df_criteria[df_criteria.criteria == "vegan"]
criteria2_group = folium.FeatureGroup(name = "vegan")
HeatMap(data = criteria2[["latitud","longitud"]],radius = 15).add_to(criteria2_group)
criteria2_group.add_to(heat_map)

criteria3 = df_criteria[df_criteria.criteria == "basketball stadium"]
criteria3_group = folium.FeatureGroup(name = "basketball stadium")
HeatMap(data = criteria3[["latitud","longitud"]],radius = 15).add_to(criteria3_group)
criteria3_group.add_to(heat_map)

criteria4 = df_criteria[df_criteria.criteria == "night_clubs"]
criteria4_group = folium.FeatureGroup(name = "night_clubs")
HeatMap(data = criteria4[["latitud","longitud"]],radius = 15).add_to(criteria4_group)
criteria4_group.add_to(heat_map)

criteria5 = df_criteria[df_criteria.criteria == "transports"]
criteria5_group = folium.FeatureGroup(name = "transports")
HeatMap(data = criteria5[["latitud","longitud"]],radius = 15).add_to(criteria5_group)
criteria5_group.add_to(heat_map)

In [97]:
icon = Icon(color = "white",
            prefix = "fa",
            icon = "building-o",
            icon_color = "blue")
loc = {"location":[office_lat,office_long],
       "tooltip": "Office Location"}

marker_office = Marker(**loc, icon = icono)
marker_office.add_to(heat_map)

In [98]:
folium.LayerControl(collapsed=False).add_to(heat_map)

In [99]:
heat_map

## Map by criterias

In [100]:
map_final = folium.Map (location = [40.7592189,-73.9783534],zoom_start = 15, min_lon = - 180, max_lon = 180, attr = 'Mapbox attribution')

In [101]:
for i,row in df_criteria.iterrows():
    name = {"location" : [row["latitud"],row["longitud"]],
            "tooltip" : row["name"]}
    
    if row["criteria"] == 'Starbucks':
        icon = Icon(color = "darkpurple",
                    prefix = "fa",
                    icon = "coffee",
                    icon_color = "beige")
    elif row["criteria"] == 'transports':
        icon = Icon(color = "blue",
                    prefix = "fa",
                    icon = "globe",
                    icon_color = "beige")
    elif row["criteria"] == 'office location':
        icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "building-o",
                    icon_color = "beige")
    elif row["criteria"] == 'basketball stadium':
        icon = Icon(color = "orange",
                    prefix = "fa",
                    icon = "futbol-o",
                    icon_color = "beige")
    elif row["criteria"] == 'vegan':
        icon = Icon(color = "green",
                    prefix = "fa",
                    icon = "cutlery",
                    icon_color = "beige")
    else:
        icon = Icon(color = "white",
                    prefix = "fa",
                    icon = "glass",
                    icon_color = "lightblue")
        
    Marker(**name,icon = icon).add_to(map_final)

In [102]:
map_final

## CONCLUSION

The office **location** proposal, located in NYC with **coordinates (40.7592189, -73.9783534)**, satisfies 5 criterias of the suggested ones.
* A sum of **20 Starbucks** are located in a radius of 500m from the office. 
  * The closest ones coordinates are:(40.756086423412924, -73.9749708319223).
  
  
* **30 Vegan restaurants** are located in a radius of 500m from the office. 
  * Nearest one is called 'Devon & Blakely' and its coordinates are (40.75560132228682, -73.97619031651554).
  
  
* **4 basketball stadiums** are located in a radius of 1000m. **Up to 22 increasing** the radius to 10km. 
  * Nearestt stadium: ***Zaro's Bakery***, (40.752875, -73.976946).
  
  
* A total amount of **11 night clubs** in a radius of 1000m from the office. 
  * ***Public House*** is the closest one and its coordinates are:(40.75055411817874,-73.97603942215522).
  
  
* **9 transports points** close to the office that facilitates travel for Account Managers. 
  * The closest transport location is Active Transport ServicesInc, (40.67378371119131,-73.98247150733843).